In [ ]:
# Mechanism to mock test submits in MongoDB with random dates.
from bson import ObjectId
from config.database import db_UJ        # UJ mongoDB
from datetime import datetime, timedelta
from pymongo import UpdateOne
import random

class MockTestSubmits:
    def __init__(self):
        self.cts = db_UJ['test_st']

    def random_date_gen(self, simplify=False, total_dates=5):
        def df(dt):
            return dt.strftime("%m-%d")

        now = datetime.now()  # Get current date and time
        dates = []

        # Generate dates for the last 3 days (today and the two previous days)
        for i in range(3):
            date = now - timedelta(days=i)
            dates.append(date)

        # Randomly select 5 items from these dates (with replacement)
        random_items = random.choices(dates, k=random.randint(1, total_dates))
        # If simplify is True, format the dates
        if simplify:
            # Format the dates using your df function
            formatted_items = [df(item) for item in random_items]
            return formatted_items    
        else:
            return random_items
        # Format the dates using your df function
        # formatted_items = [df(item) for item in random_items]
        # return formatted_items

    def update_operations(self,mdb_set):
        cts = db_UJ['test_st']   # create/get a collection
        doc_ids = list(cts.find({},{"_id":1}))       # Get limited test document IDs
        update_operation = []
        for i in doc_ids:
            juice = {'submits':self.random_date_gen(simplify=False, total_dates=15)}
            # Use this to remove all the submits fields
            # Use this to simulate setting the new submits field with mock timestamps
            set_ops = {'$set': juice} if mdb_set else {'$unset': {'submits': ''}}
            update_operation.append(
                UpdateOne(
                {"_id": ObjectId(i['_id'])},
                set_ops
            ))
        return update_operation

    def update_submits(self, mdb_set):
        self.cts.bulk_write(self.update_operations(mdb_set=mdb_set))

    def check_transformed_submits(self):
        original = list(self.cts.find({'submits': {'$exists': True}},{"_id":0,"ph":0,"r_id":0}))
        transformed = [{v1: v2} for d in original for v1, v2 in zip(d.values(), list(d.values())[1:])]
        return transformed

In [ ]:
mts = MockTestSubmits()
# mts.update_operations(mdb_set=False)
# mts.update_submits(mdb_set=False)


In [53]:
ctrs = db_UJ['test_rst']   # create/get a collection
submitTerm = "asdf"
timestamp = datetime.now()  # Get current date and time
ctrs.update_one(
    {"rst": submitTerm},
    {"$push": {"submits": timestamp}},
    upsert=True
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000001c0'), 'opTime': {'ts': Timestamp(1747987089, 18), 't': 448}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747987089, 18), 'signature': {'hash': b'\t^\xce\x93pp\x7f\x1b\x90\xd2\x8b\xf9\x9f6\x19O\x11\xec\xb9O', 'keyId': 7460207444094877709}}, 'operationTime': Timestamp(1747987089, 18), 'updatedExisting': True}, acknowledged=True)

In [ ]:
# list(ctrs.find({}).limit(5))
# ctrs.delete_many({})

DeleteResult({'n': 3, 'electionId': ObjectId('7fffffff00000000000001c0'), 'opTime': {'ts': Timestamp(1747992429, 3), 't': 448}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1747992429, 3), 'signature': {'hash': b'w\xf4\x98J\xde\xba\x98Hso\xa8-EN7m\x95.\x12`', 'keyId': 7460207444094877709}}, 'operationTime': Timestamp(1747992429, 3)}, acknowledged=True)

In [ ]:
a = ctrs.aggregate([
    { "$match": { "submits": { "$exists": True} } },
    { "$unwind": "$submits" },
    { "$project": {
        "_id": 0,
        "rst": 1,
        "timestamp": "$submits"
    }}
])
# list(a)
ctrs.find_one({})


In [ ]:
cts_returns =  list(mts.cts.aggregate([
        { "$match": { "submits": { "$exists": True} } },
        { "$unwind": "$submits" },
        { "$addFields": { "timestamp": "$submits" } },
        { "$unset": ["_id", "r_id", "ph", "submits"] }
    ]))

In [ ]:
list(mts.cts.find({"submits": {"$exists": True}}))